In [4]:
import pandas as pd

df = pd.read_csv("../data/cleaned/online_retail_cleaned.csv")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12


In [ ]:
first_purchase = (
    df.groupby('CustomerID')['InvoiceMonth']
    .min()
    .reset_index(name='CohortMonth')
)

df=df.merge(first_purchase, on='CustomerID', how = 'left')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,InvoiceMonth,CohortMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,2010-12


In [6]:
df['CohortIndex'] = (
    (df['InvoiceMonth'].dt.year - df['CohortMonth'].dt.year) * 12 +
    (df['InvoiceMonth'].dt.month - df['CohortMonth'].dt.month)
)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,InvoiceMonth,CohortMonth,CohortIndex
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12,2010-12,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,2010-12,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12,2010-12,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,2010-12,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,2010-12,0


In [ ]:
cohort_revenue = (
    df.groupby(['CohortMonth', 'CohortIndex'])['Revenue']
    .sum()
    .reset_index()
)


cohort_revenue.head()

,CohortMonth,CohortIndex,Revenue
0,2010-12,0,572713.89
1,2010-12,1,276237.69
2,2010-12,2,233845.37
3,2010-12,3,303119.39
4,2010-12,4,204407.66


In [8]:
cohort_revenue_pivot = cohort_revenue.pivot(
    index='CohortMonth',
    columns='CohortIndex',
    values='Revenue'
)

In [16]:
cohort_cumulative_revenue = cohort_revenue_pivot.cumsum(axis=1)
cohort_cumulative_revenue.round(2)

CohortIndex,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12,572713.89,848951.58,1082796.95,1385916.34,1590324.00,1926951.50,2241114.11,2551898.03,2883647.25,3356460.89,3812541.50,4326345.09,4512148.22
2011-01,293207.35,348357.26,411625.43,483212.83,564326.92,648910.70,719018.10,791707.81,863731.46,975535.36,1099417.46,1125882.28,NaN
2011-02,158142.07,187185.31,228319.69,276558.55,316706.46,350929.69,400636.32,463087.15,518467.21,583276.66,593876.88,NaN,NaN
2011-03,200069.96,230129.39,289186.90,332003.51,383660.84,423681.65,488601.14,559625.09,630890.92,643758.94,NaN,NaN,NaN
2011-04,122011.49,151464.31,176541.38,200852.85,227167.04,257373.34,286042.32,320270.93,326621.03,NaN,NaN,NaN,NaN
2011-05,124103.78,142786.54,162980.30,182142.47,210004.34,242980.94,276293.85,455545.38,NaN,NaN,NaN,NaN,NaN
2011-06,135742.95,150548.10,164703.50,195665.09,222419.63,265227.88,273455.23,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,74008.75,85798.42,101332.60,118884.07,138396.18,144468.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,79756.10,100719.03,136222.93,180764.69,196049.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
cohort_size = (
    df.groupby('CohortMonth')['CustomerID']
    .nunique()
)

revenue_per_customer = cohort_cumulative_revenue.divide(
    cohort_size, axis=0
)

revenue_per_customer.round(2)


CohortIndex,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12,647.13,959.27,1223.50,1566.01,1796.98,2177.35,2532.33,2883.50,3258.36,3792.61,4307.96,4888.53,5098.47
2011-01,703.14,835.39,987.11,1158.78,1353.30,1556.14,1724.26,1898.58,2071.30,2339.41,2636.49,2699.96,NaN
2011-02,416.16,492.59,600.84,727.79,833.44,923.50,1054.31,1218.65,1364.39,1534.94,1562.83,NaN,NaN
2011-03,442.63,509.14,639.79,734.52,848.81,937.35,1080.98,1238.11,1395.78,1424.25,NaN,NaN,NaN
2011-04,406.70,504.88,588.47,669.51,757.22,857.91,953.47,1067.57,1088.74,NaN,NaN,NaN,NaN
2011-05,436.99,502.77,573.87,641.35,739.45,855.57,972.87,1604.03,NaN,NaN,NaN,NaN,NaN
2011-06,560.92,622.10,680.59,808.53,919.09,1095.98,1129.98,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,393.66,456.37,539.00,632.36,736.15,768.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,471.93,595.97,806.05,1069.61,1160.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
